# HMC NUTS sampling with CmdStanPy

of Montbrio model. On SDE and ODE data, when both variables are available.

In [1]:
import os
import syss
import numpy as np
import json
import matplotlib.pyplot as plt

In [2]:
pwd

'/home/tng/Documents/montbrio/benchmark_NEW'

In [3]:
cwd = os.getcwd()
cwd

'/home/tng/Documents/montbrio/benchmark_NEW'

In [4]:
#cmdstanpy.install_cmdstan()

In [5]:
#Setting the path for cmdstan
import cmdstanpy
cmdstanpy.set_cmdstan_path('/home/tng/.cmdstan/cmdstan-2.31.0')
cmdstanpath = cmdstanpy.cmdstan_path()

In [6]:
def RMSE(x1, x2):
    return np.sqrt(((x1 - x2) ** 2).mean())

In [ ]:
stan_models_dir = cwd + '/stan_models/'

### SDE data with dynamical noise noise 0.1

In [7]:
sigma = 0.1
data_input_filename=str('data_input_mpr_sd'+ '_sigma='+str(sigma))
data_input_path = cwd+'/Res_syntheticData/data_input_files/'+ str(data_input_filename)+ '.R.npz'
data_input = dict(np.load(data_input_path))
v_obs, r_obs = data_input['vs'], data_input['rs']
X_obs = np.vstack((v_obs, r_obs))
sigma_noise = sigma

In [8]:
model='uniform_fixedsig'

In [ ]:
print("sigma=", sigma_noise)
data = data_input.copy()
data['rs'] = X_obs[1, :]
data['vs'] = X_obs[0, :]
data['ds'] = 1 #no downsampling

fit_folder = cwd + "/output_HMC/" + "output_hmc_" + model +'_noise=' + str(sigma_noise) 
cmdModel = cmdstanpy.CmdStanModel(stan_file=stan_models_dir + model + '.stan')
cmdModel.compile()
fit = cmdModel.sample(data=data, iter_warmup=1000, iter_sampling=500, save_warmup=False,
                      chains=4,
                      show_progress=True, output_dir=fit_folder, seed=0)

Load the output file and extract interest quantities

In [ ]:
! mkdir -p $cwd/results

In [ ]:
print('sigma=', str(sigma_noise))

fit_folder = cwd + "/output_HMC/" + "output_hmc_" + model +'_noise=' + str(sigma_noise)
N_chains = 4
chains = []
for i in range(N_chains) : 
    chain_i = dict()
    fit = cmdstanpy.from_csv(fit_folder + '/' + model + '*' + str(i+1) + '.csv') 
    for variable in ['delta', 'eta', 'J', 'eps'] :
        chain_i[variable] = fit.stan_variable(variable)
    chains.append(chain_i)

params_true = np.array([data_input['delta_true'], data_input['eta_true'], data_input['J_true']])

params_est_distrib = []
for chain in chains :
    params_est_distrib.append(np.array([chain['delta'], chain['eta'], chain['J'], chain['eps']]))
params_est_distrib = np.array(params_est_distrib) 

res_prms_dict = {'delta_distrib': list(params_est_distrib[:, 0, :].mean(axis=0)), 
                 'eta_distrib': list(params_est_distrib[:, 1, :].mean(axis=0)),
                 'J_distrib': list(params_est_distrib[:, 2, :].mean(axis=0)),
                 'eps_dsitrib': list(params_est_distrib[:, 3, :].mean(axis=0))
                }
json_object = json.dumps(res_prms_dict, indent=4)
with open("results/hmc_sim_prmsdistrib_output.json", "w") as outfile:
    outfile.write(json_object)


In [ ]:
with open("results/hmc_sim_prmsdistrib_output.json", "r") as openfile:
    sim = json.load(openfile)

In [ ]:
theta_posterior = np.vstack([sim['delta_distrib'], sim['eta_distrib'], sim['J_distrib'], sim['eps_distrib']]).T

In [ ]:
from report_metrics import posterior_peaks
var_labels = ['delta', 'eta', 'J', 'epsilon']
MAP = posterior_peaks(torch.as_tensor(theta_posterior), return_dict=True, 
                      labels=var_labels)
MAP_vals = list(MAP.values())
MAP_val
params_est = MAP_vals[:-1]

In [ ]:
rmse_prms = RMSE(params_true, params_est)
rmse_prms

In [ ]:
for i in range(len(var_labels)) :
    sim[var_labels[i]] = MAP_vals[i]
sim['rmse_prms'] = rmse_prms

json_object = json.dumps(sim, indent=4)
with open("results/hmc_sim_prms_output.json", "w") as outfile:
    outfile.write(json_object)

### ODE data 

In [ ]:
sigma = 0.0
data_input_filename=str('data_input_mpr_sd'+ '_sigma='+str(sigma))
data_input_path = cwd+'/Res_syntheticData/data_input_files/'+ str(data_input_filename)+ '.R.npz'
data_input = dict(np.load(data_input_path))
v_obs, r_obs = data_input['vs'], data_input['rs']
X_obs = np.vstack((v_obs, r_obs))
sigma_noise = sigma

In [ ]:
model='uniform_ODE'

In [ ]:
print("sigma=", sigma_noise)
data = data_input.copy()
data['rs'] = X_obs[1, :]
data['vs'] = X_obs[0, :]
data['ds'] = 1 #no downsampling

fit_folder = cwd + "/output_HMC/" + "output_hmc_" + model +'_noise=' + str(sigma_noise) 
cmdModel = cmdstanpy.CmdStanModel(stan_file=stan_models_dir + model + '.stan')
cmdModel.compile()
fit = cmdModel.sample(data=data, iter_warmup=1000, iter_sampling=500, save_warmup=False,
                      chains=4,
                      show_progress=True, output_dir=fit_folder, seed=0)